In [0]:
import numpy as np

# Generacion de las matrices

#m1 = (1,2,3)
#m2 = (1,2,3)

#m1 = ((1,2),(3,4),(5,6))
#m2 = ((1,2),(3,4),(5,6))

#m1 = ((1,2,3),(4,5,6))
#m2 = ((1,2,3),(4,5,6))

m1 = ((1,2),(3,4))
m2 = ((1,2),(3,4))

In [0]:
# Creamos el RDD 
rdd1 = spark.sparkContext.parallelize((m1,m2))

print(rdd1.collect())

[((1, 2), (3, 4)), ((1, 2), (3, 4))]

In [0]:
# Uso de flat map para asignar el indice a cada fila
rdd2 = rdd1.flatMap(lambda x: [(y, x[y]) for y in range(0, len(x))])

print(rdd2.collect())

[(0, (1, 2)), (1, (3, 4)), (0, (1, 2)), (1, (3, 4))]

In [0]:
import functools

def suma(a,b):
  t = []
  l = len(a)
  for x in range(0,l):
    t.append(a[x] + b[x])
  return tuple(t)

def resta(a,b):
  t = []
  l = len(a)
  for x in range(0,l):
    t.append(a[x] - b[x])
  return tuple(t)


# Para printear la matriz final resultante
def printMatrix(m):
  for (x,y) in m: print(y)

In [0]:
#Operacion final juntando los resultados mediante reduceKey y la operacion seleccionada

rdd3 = rdd2.reduceByKey(lambda a,b: suma(a,b))

for x in m1: print(x)
print( ' + ')
for x in m2: print(x)
print( ' = ')
printMatrix(rdd3.collect())

(1, 2)
(3, 4)
 + 
(1, 2)
(3, 4)
 = 
(2, 4)
(6, 8)

In [0]:
rdd4 = rdd2.reduceByKey(lambda a,b: resta(a,b))

for x in m1: print(x)
print( ' - ')
for x in m2: print(x)
print( ' = ')
printMatrix(rdd4.collect())

(1, 2)
(3, 4)
 - 
(1, 2)
(3, 4)
 = 
(0, 0)
(0, 0)

In [0]:
# Para que se puedan multiplicar:  colums de la primera == a las filas de la segunda

def mult(x,y):
  fila_x = x[0]
  fila_y = y[0]
  


rdd5 = rdd2.flatMap(lambda x,y: mult(x,y))

for x in m1: print(x)
print( ' * ')
for x in m2: print(x)
print( ' = ')
printMatrix(rdd5.collect())

(1, 2)
(3, 4)
 * 
(1, 2)
(3, 4)
 =

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1544505096871866> in <module> 
 11 for x in m2 : print ( x ) 
 12 print ( ' = ' ) 
 ---> 13 printMatrix ( rdd5 . collect ( ) ) 

 /databricks/spark/python/pyspark/rdd.py in collect (self) 
 965 # Default path used in OSS Spark / for non-credential passthrough clusters: 
 966 with SCCallSiteSync ( self . context ) as css : 
 --> 967 sock_info = self . ctx . _jvm . PythonRDD . collectAndServe ( self . _jrdd . rdd ( ) ) 
 968 return list ( _load_from_socket ( sock_info , self . _jrdd_deserializer ) ) 
 969 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 209.0 failed 1 times, most recent failure: Lost task 3.0 in stage 209.0 (TID 1595) (ip-10-172-231-59.us-west-2.compute.internal executor driver): org.apache.spark.api.python.PythonException: 'TypeError: <lambda>() missing 1 required positional argument: 'y''. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 701, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 693, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 267, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/util.py", line 73, in wrapper
 return f(*args, **kwargs)
TypeError: <lambda>() missing 1 required positional argument: 'y'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:661)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:813)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:795)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:614)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at org.apache.spark.InterruptibleIterator.foreach(InterruptibleIterator.scala:28)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:105)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:49)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at org.apache.spark.InterruptibleIterator.to(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce.toBuffer(TraversableOnce.scala:307)
	at scala.collection.TraversableOnce.toBuffer$(TraversableOnce.scala:307)
	at org.apache.spark.InterruptibleIterator.toBuffer(InterruptibleIterator.scala:28)
	at scala.collection.TraversableOnce.toArray

In [0]:
# Division = A / B = A * B ^ -1
# B ^ 1 = (1/Determinante de B) * Transpuesta de la matriz adjunta a B